# C- Wind Turbine Induction Generator Design

## Project Specifications

In this project, a squirrel cage induction generator for the Northerl Energy's VIRA-250 wind turbine will be designed. The specifications are as follows:

Rated Power: 250 kW
Rated Wind Speed: 14 m/s
Rated Turbine Speed: 24.3 rpm
Gear Ratio: 31.2
Number of Poles: 8
Line to line voltage: 400 V
Frequency: 50 Hz
Rated Speed: 758 rpm
Gearbox: (Coupled from wind turbine blade)
Insulation Class: F